In [1]:
pip install chromadb sentence-transformers

  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached pyreadline3-3.5.4-py3-none-any.whl.metadata (4.7 kB)
   ---------------------------------------- 0.0/19.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.8 MB ? eta -:--:--
    --------------------------------------- 0.3/19.8 MB ? eta -:--:--
    --------------------------------------- 0.3/19.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/19.8 MB 622.7 kB/s eta 0:00:32
   - -------------------------------------- 0.5/19.8 MB 622.7 kB/s eta 0:00:32
   - -

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azure-monitor-opentelemetry 1.6.10 requires opentelemetry-sdk<1.32,>=1.28.0, but you have opentelemetry-sdk 1.37.0 which is incompatible.
opentelemetry-instrumentation 0.52b1 requires opentelemetry-semantic-conventions==0.52b1, but you have opentelemetry-semantic-conventions 0.58b0 which is incompatible.
opentelemetry-instrumentation-asgi 0.52b1 requires opentelemetry-semantic-conventions==0.52b1, but you have opentelemetry-semantic-conventions 0.58b0 which is incompatible.
opentelemetry-instrumentation-dbapi 0.52b1 requires opentelemetry-semantic-conventions==0.52b1, but you have opentelemetry-semantic-conventions 0.58b0 which is incompatible.
opentelemetry-instrumentation-django 0.52b1 requires opentelemetry-semantic-conventions==0.52b1, but you have opentelemetry-semantic-conventions 0.58b0 which is incompatibl

In [3]:
import json
import os
from statistics import median

# --- NEW: An object to store all analysis results for a single log file ---
class SparkLogAnalysis:
    """A container object for all parsed summaries from a Spark event log."""
    def __init__(self, log_file_path):
        self.application_id = "unknown_app"
        self.log_file = os.path.basename(log_file_path)
        self.job_summaries = []
        self.stage_summaries = []
        # Use a dictionary for unique, application-wide summaries
        self.application_summaries = {}

    def set_application_id(self, app_id):
        if app_id:
            self.application_id = app_id

    def add_summary(self, summary_dict):
        """Categorizes and adds a summary dictionary to the appropriate list."""
        if not summary_dict:
            return
            
        event_type = summary_dict.get("event_type")
        # Ensure the application ID is consistent
        summary_dict["application_id"] = self.application_id

        if event_type == "JobPerformanceSummary":
            self.job_summaries.append(summary_dict)
        elif event_type == "StagePerformanceSummary":
            self.stage_summaries.append(summary_dict)
        else: # For DiskFailure, Blacklist, Churn summaries
            self.application_summaries[event_type] = summary_dict

    def to_dict(self):
        """Converts the entire analysis object to a dictionary."""
        return {
            "application_id": self.application_id,
            "log_file": self.log_file,
            "job_summaries": self.job_summaries,
            "stage_summaries": self.stage_summaries,
            "application_summaries": self.application_summaries
        }

    def __repr__(self):
        """Provides a clean, printable JSON representation of the object."""
        return json.dumps(self.to_dict(), indent=2)


def analyze_spark_log(log_file_path):
    """
    Parses a single Spark event log and returns a single SparkLogAnalysis object.
    """
    # 1. Create the container object at the start
    analysis_object = SparkLogAnalysis(log_file_path)
    
    # --- Data structures for parsing (unchanged) ---
    stage_data = {}
    job_data = {}
    executor_lifecycle = {}
    disk_related_executor_failures = {}
    blacklisted_executors = []

    with open(log_file_path, 'r') as f:
        for line in f:
            try:
                event = json.loads(line)
            except json.JSONDecodeError:
                continue

            event_type = event.get("Event")
            
            if event_type == "SparkListenerApplicationStart":
                # Set the application ID for the entire object
                analysis_object.set_application_id(event.get("App ID"))
            
            # --- The rest of the parsing logic is the same, but uses `add_summary` instead of `yield` ---
            elif event_type == "SparkListenerExecutorAdded":
                executor_id, timestamp = event.get("Executor ID"), event.get("Timestamp")
                executor_lifecycle.setdefault(executor_id, {})['add_time'] = timestamp

            elif event_type == "SparkListenerExecutorRemoved":
                executor_id, timestamp = event.get("Executor ID"), event.get("Timestamp")
                reason = event.get("Removed Reason", "")
                executor_lifecycle.setdefault(executor_id, {})['remove_time'] = timestamp
                if "no space left on device" in reason.lower() or "disk" in reason.lower():
                    disk_related_executor_failures.setdefault(reason, {"count": 0, "samples": []})
                    disk_related_executor_failures[reason]["count"] += 1
                    if len(disk_related_executor_failures[reason]["samples"]) < 3:
                        disk_related_executor_failures[reason]["samples"].append({"executor_id": executor_id, "timestamp": timestamp})

            elif event_type == "SparkListenerExecutorBlacklisted":
                blacklisted_executors.append({"executor_id": event.get("Executor ID"), "timestamp": event.get("Timestamp"), "reason": f"Blacklisted for {event.get('Task ID')} task failures"})

            elif event_type == "SparkListenerJobStart":
                job_id = event.get("Job ID")
                job_data[job_id] = {"start_time": event.get("Submission Time")}

            elif event_type == "SparkListenerJobEnd":
                job_id = event.get("Job ID")
                if job_id in job_data:
                    duration_ms = event.get("Completion Time") - job_data[job_id]["start_time"]
                    job_result = event.get("Job Result", {}).get("Result")
                    # 2. Add job summary to the object
                    analysis_object.add_summary({ "event_type": "JobPerformanceSummary", "job_id": job_id, "status": job_result, "duration_s": duration_ms / 1000, "summary": f"Job {job_id} finished with status {job_result} in {duration_ms / 1000:.2f} seconds." })
                    del job_data[job_id]

            elif event_type == "SparkListenerTaskEnd":
                stage_id = event.get("Stage ID")
                if stage_id is not None:
                    stage_data.setdefault(stage_id, {"tasks": [], "failed_task_count": 0})
                    if event.get("Task End Reason", {}).get("Reason") != "Success": stage_data[stage_id]["failed_task_count"] += 1
                    task_info, task_metrics = event.get("Task Info", {}), event.get("Task Metrics", {})
                    duration_ms = task_info.get("Finish Time", 0) - task_info.get("Launch Time", 0)
                    stage_data[stage_id]["tasks"].append({ "duration_ms": duration_ms, "jvm_gc_time_ms": task_metrics.get("JVM GC Time", 0), "executor_run_time_ms": task_metrics.get("Executor Run Time", 1), "executor_deserialize_time_ms": task_metrics.get("Executor Deserialize Time", 0), "shuffle_fetch_wait_time_ms": task_metrics.get("Shuffle Read Metrics", {}).get("Fetch Wait Time", 0), "memory_spilled_bytes": task_metrics.get("Memory Bytes Spilled", 0), "disk_spilled_bytes": task_metrics.get("Disk Bytes Spilled", 0), })

            elif event_type == "SparkListenerStageCompleted":
                stage_id = event.get("Stage Info", {}).get("Stage ID")
                if stage_id in stage_data and len(stage_data[stage_id]["tasks"]) > 0:
                    tasks, failed_task_count = stage_data[stage_id]["tasks"], stage_data[stage_id]["failed_task_count"]
                    task_durations = [t["duration_ms"] for t in tasks]
                    total_duration, total_gc_time, total_runtime = sum(task_durations), sum(t["jvm_gc_time_ms"] for t in tasks), sum(t["executor_run_time_ms"] for t in tasks)
                    total_deserialize_time, total_fetch_wait_time = sum(t["executor_deserialize_time_ms"] for t in tasks), sum(t["shuffle_fetch_wait_time_ms"] for t in tasks)
                    total_spilled_bytes = sum(t["memory_spilled_bytes"] for t in tasks) + sum(t["disk_spilled_bytes"] for t in tasks)
                    summary_parts, potential_issues = [f"Stage {stage_id} completed with {len(tasks)} tasks."], []
                    if failed_task_count > 0: summary_parts.append(f"It experienced {failed_task_count} task failures that required retries."); potential_issues.append("TASK_FAILURES")
                    max_duration, median_duration = max(task_durations), median(task_durations)
                    if max_duration > 3 * median_duration and max_duration > 20000: potential_issues.append("DATA_SKEW"); summary_parts.append(f"Detected potential data skew. Max task duration: {max_duration/1000:.2f}s, median: {median_duration/1000:.2f}s.")
                    if total_runtime > 0 and (total_gc_time / total_runtime) > 0.10: potential_issues.append("HIGH_GC_PRESSURE"); summary_parts.append(f"High JVM GC pressure detected ({ (total_gc_time / total_runtime) * 100:.1f}% of runtime).")
                    if total_spilled_bytes > 0: potential_issues.append("DATA_SPILL"); summary_parts.append(f"Detected data spilling to disk ({total_spilled_bytes / (1024*1024):.2f} MB).")
                    if total_duration > 0 and (total_fetch_wait_time / total_duration) > 0.25: potential_issues.append("SHUFFLE_BOTTLENECK"); summary_parts.append(f"Significant shuffle bottleneck, tasks spent {(total_fetch_wait_time / total_duration) * 100:.1f}% of time waiting for data.")
                    if total_runtime > 0 and (total_deserialize_time / total_runtime) > 0.15: potential_issues.append("DESERIALIZATION_BOTTLENECK"); summary_parts.append(f"High deserialization time ({(total_deserialize_time / total_runtime) * 100:.1f}% of runtime).")
                    # 3. Add stage summary to the object
                    analysis_object.add_summary({ "event_type": "StagePerformanceSummary", "stage_id": stage_id, "metrics": { "task_count": len(tasks), "failed_task_count": failed_task_count, "max_task_duration_ms": max_duration, "median_task_duration_ms": median_duration, "total_spilled_mb": total_spilled_bytes / (1024*1024) }, "summary": " ".join(summary_parts), "potential_issues": potential_issues if potential_issues else ["NONE"] })

    # --- Generate and Add Final Summaries ---
    if disk_related_executor_failures:
        total_failures = sum(v['count'] for v in disk_related_executor_failures.values())
        analysis_object.add_summary({ "event_type": "ExecutorDiskFailureSummary", "total_failures": total_failures, "summary": f"A critical error occurred where {total_failures} executors failed to launch due to running out of disk space on the nodes.", "failures_by_reason": disk_related_executor_failures })
    if blacklisted_executors:
        analysis_object.add_summary({ "event_type": "ExecutorBlacklistSummary", "total_blacklisted": len(blacklisted_executors), "blacklisted_executors": blacklisted_executors })
    short_lived_executors = []
    churn_threshold_ms = 60000 
    for exec_id, times in executor_lifecycle.items():
        if 'add_time' in times and 'remove_time' in times and (times['remove_time'] - times['add_time']) < churn_threshold_ms:
            short_lived_executors.append({"executor_id": exec_id, "lifespan_s": (times['remove_time'] - times['add_time']) / 1000})
    if short_lived_executors:
        avg_lifespan_s = sum(e['lifespan_s'] for e in short_lived_executors) / len(short_lived_executors)
        analysis_object.add_summary({ "event_type": "ExecutorChurnSummary", "total_churn_events": len(short_lived_executors), "summary": f"Detected {len(short_lived_executors)} executors with an average lifespan of just {avg_lifespan_s:.2f}s, indicating severe instability.", "average_lifespan_s": round(avg_lifespan_s, 2), "sample_churned_executors": short_lived_executors[:3] })

    # 4. Return the single, populated object
    return analysis_object


# --- MODIFIED: Main execution block to handle the returned object ---
if __name__ == "__main__":
    log_directory = r"C:\Users\ddev\Documents\projects\spark-event-logs\test-logs"

    if not os.path.isdir(log_directory):
        print(f"Error: Directory not found at '{log_directory}'. Please update the path.")
    else:
        print(f"--- Starting analysis of logs in: {log_directory} ---")
        all_analyses = [] # A list to hold all the final analysis objects
        for filename in os.listdir(log_directory):
            full_file_path = os.path.join(log_directory, filename)
            if os.path.isfile(full_file_path):
                print(f"\n--- Processing file: {filename} ---")
                try:
                    # The function now returns a single object
                    final_analysis_object = analyze_spark_log(full_file_path)
                    
                    # You can now pass this object to another function,
                    # or just print it for verification.
                    print(final_analysis_object)
                    
                    # Optionally, add it to a list for further processing
                    all_analyses.append(final_analysis_object)
                    
                except Exception as e:
                    print(f"Could not process file {filename}. Error: {e}")
        print("\n--- Analysis complete. ---")
        # Now you have a list 'all_analyses' where each item is an object
        # representing the full analysis of one log file.
        # print(f"Successfully parsed {len(all_analyses)} log files.")

--- Starting analysis of logs in: C:\Users\ddev\Documents\projects\spark-event-logs\test-logs ---

--- Processing file: app-20211011150934-6406 ---
{
  "application_id": "app-20211011150934-6406",
  "log_file": "app-20211011150934-6406",
  "job_summaries": [
    {
      "event_type": "JobPerformanceSummary",
      "job_id": 0,
      "status": "JobSucceeded",
      "duration_s": 2.664,
      "summary": "Job 0 finished with status JobSucceeded in 2.66 seconds.",
      "application_id": "app-20211011150934-6406"
    },
    {
      "event_type": "JobPerformanceSummary",
      "job_id": 1,
      "status": "JobSucceeded",
      "duration_s": 2.269,
      "summary": "Job 1 finished with status JobSucceeded in 2.27 seconds.",
      "application_id": "app-20211011150934-6406"
    },
    {
      "event_type": "JobPerformanceSummary",
      "job_id": 2,
      "status": "JobSucceeded",
      "duration_s": 0.768,
      "summary": "Job 2 finished with status JobSucceeded in 0.77 seconds.",
      "ap

In [5]:
import chromadb
from sentence_transformers import SentenceTransformer

# Assume 'all_analyses' is a list of the SparkLogAnalysis objects
# you created with the previous script.

# 1. Flatten the analysis objects into a single list of documents
all_summaries = []
for analysis_object in all_analyses:
    # Each of these is a list of dictionaries
    all_summaries.extend(analysis_object.job_summaries)
    all_summaries.extend(analysis_object.stage_summaries)
    # This is a dictionary, so we take its values
    all_summaries.extend(analysis_object.application_summaries.values())

# 2. Initialize the embedding model and ChromaDB client
model = SentenceTransformer('all-MiniLM-L6-v2') 
client = chromadb.Client() # For in-memory storage

# Create a collection to store your logs
collection = client.get_or_create_collection(name="spark_logs")

# 3. Prepare the data for the vector database
documents_to_embed = []
metadatas = []
ids = []

print("Processing and preparing documents...")
# Loop through the flattened list of summary dictionaries
for i, summary_doc in enumerate(all_summaries):
    # The 'summary' field is the text we will embed and search over.
    documents_to_embed.append(summary_doc["summary"])

    # All other fields become the metadata for filtering.
    metadata_payload = summary_doc.copy()
    
    # The text itself is not needed in the metadata, so we remove it.
    del metadata_payload["summary"]

    # ChromaDB metadata values must be simple types (str, int, float, bool).
    # We convert or remove complex types like lists and dicts.
    if 'potential_issues' in metadata_payload:
      # Convert list to a comma-separated string for filtering
      metadata_payload['potential_issues'] = ", ".join(metadata_payload['potential_issues'])
    if 'metrics' in metadata_payload:
      del metadata_payload['metrics'] # Removing complex dict
    if 'failures_by_reason' in metadata_payload:
      del metadata_payload['failures_by_reason'] # Removing complex dict
    if 'sample_churned_executors' in metadata_payload:
      del metadata_payload['sample_churned_executors'] # Removing list of dicts

    metadatas.append(metadata_payload)
    
    # Each document needs a unique ID.
    ids.append(f"log_{summary_doc['application_id']}_{i}")

print("Documents prepared. Adding to the vector database...")
# 4. Add the data to the collection
collection.add(
    documents=documents_to_embed,
    metadatas=metadatas,
    ids=ids
)

print("\n✅ Data successfully loaded into ChromaDB!")
print(f"Total documents in collection: {collection.count()}")

# 5. Query the data using both semantic search and metadata filtering
print("\n--- Query Example ---")

# This query will ONLY search within documents matching the application_id.
# It semantically searches for summaries related to "disk problems".
results = collection.query(
    query_texts=["What were the disk problems?"],
    n_results=1,
    where={"application_id": "app-20211011153740-6439"}
)

print("\nQuery: 'What were the disk problems?' for app 'app-20211011153740-6439'")
print("Results:")
# The 'documents' field in the result contains the original summary text.
print(results['documents'])

Processing and preparing documents...
Documents prepared. Adding to the vector database...

✅ Data successfully loaded into ChromaDB!
Total documents in collection: 789

--- Query Example ---

Query: 'What were the disk problems?' for app 'app-20211011153740-6439'
Results:
[['A critical error occurred where 54 executors failed to launch due to running out of disk space on the nodes.']]


In [23]:
results2 = collection.query(
    query_texts=["which application had most failures ? ?"],
    n_results=1
)

print("\nQuery: 'Which job ran for the longest time'")
print("Results:")
# The 'documents' field in the result contains the original summary text.
print(results2['documents'])


Query: 'Which job ran for the longest time'
Results:
[['A critical error occurred where 39 executors failed to launch due to running out of disk space on the nodes.']]
